<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/sem_foro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.title("Dashboard Semáforo de Fechas PO vs EXW Real")
st.write("Carga tu archivo Excel para visualizar el semáforo de fechas:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')

    # Filtrar solo filas con EXW Real
    df_validas = df[df['EXW Real'].notna()].copy()

    # Calcular diferencia en días
    df_validas['Diferencia días calculada'] = (df_validas['EXW Real'] - df_validas['Fecha de PO']).dt.days

    # Semáforo
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif 0 <= dias <= 6:
            return 'En tiempo'
        elif dias >= 7:
            return 'On Delay'
        else:
            return 'Revisar'
    df_validas['Semáforo'] = df_validas['Diferencia días calculada'].apply(status_semáforo)

    # Mostrar tabla
    st.subheader("Tabla con Semáforo (solo registros con EXW Real)")
    st.dataframe(df_validas[['Fecha de PO', 'EXW Real', 'Diferencia días calculada', 'Semáforo']])

    # Gauge chart: mostrar promedio de la diferencia de días
    st.subheader("Indicador Gauge")

    # Calcula el promedio
    if not df_validas['Diferencia días calculada'].dropna().empty:
        promedio_dias = df_validas['Diferencia días calculada'].mean()
    else:
        promedio_dias = 0

    # Define límites para el gauge (puedes personalizar)
    minimo = 0
    maximo = max(14, df_validas['Diferencia días calculada'].max() or 14)  # mínimo 14 días como rango

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = promedio_dias,
        number = {'suffix': " días", 'font': {'size': 36}},
        title = {'text': "Indicador Gauge"},
        gauge = {
            'axis': {'range': [minimo, maximo], 'tickmode': 'linear'},
            'steps': [
                {'range': [minimo, 6], 'color': "#ABEBC6"},    # verde
                {'range': [6, 14], 'color': "#F9E79F"},        # amarillo
                {'range': [14, maximo], 'color': "#F5B7B1"}    # rojo
            ],
            'bar': {'color': "black"}  # Manecilla en negro
        }
    ))

    st.plotly_chart(fig)

    st.info("Dashboard generado por Isra1 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")

